In [ ]:
from openexr.spectralexr import SpectralEXRFile
from radiometry.cmf import CMF
from radiometry.spectrum import Spectrum

import compress.moments as mnt
import compress.util as mntutil 

import os
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
def show_spectral_reflective_images(
    cmf: CMF, illu: Spectrum,
    titles,
    spectral_wavelengths,
    spectral_images):
    fig, ax = plt.subplots(1, len(spectral_images))

    for a, title, wl, spec_img in zip(ax, titles, spectral_wavelengths, spectral_images):
        rgb_image = np.clip(cmf.get_sRGB_lin_reflective_img(
            illu.data[:,0], illu.data[:,1],
            wl, spec_img
        ), 0, 1)

        a.set_title(title)
        a.set_xticks([])
        a.set_yticks([])
        a.imshow(rgb_image**(1./2.2))

    fig.tight_layout()
    plt.show()

# Load an image and misc. assets

In [ ]:
image_path = "/home/afichet/Repositories/spectral-compress/build/macbeth.exr"

cmf_cie_2006_2 = CMF(os.path.join('..', 'data', 'cmf', 'ciexyz06_2deg.csv'))
illu_d65 = Spectrum(os.path.join('..', 'data', 'spectra', 'illuminant_D65.csv'))

In [ ]:
image_in = SpectralEXRFile(image_path)

wavelengths = image_in.reflective_wavelengths_nm
spectral_image = image_in.reflective_image

In [ ]:
rgb_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    wavelengths, spectral_image
)

plt.plot()
plt.xticks([])
plt.yticks([])
plt.imshow(rgb_image**(1./2.2))
plt.tight_layout()
plt.show()

# Compression pipeline

## Compression

### Current

#### Bounded

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
basis  = mnt.get_basis_signal_to_moments(phases)

compressed_bounded_moment_image, mins, maxs = mnt.bounded_forward(basis, spectral_image)

#### Unbounded

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
basis  = mnt.get_basis_signal_to_moments(phases)

compressed_unbounded_moment_image, mins, maxs = mnt.unbounded_forward(basis, spectral_image)

#### Unbounded to bounded

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
basis  = mnt.get_basis_signal_to_moments(phases)

compressed_unbounded_to_bounded_moment_image, mins, maxs = mnt.unbounded_to_bounded_forward(basis, spectral_image)

### Alternative (bounded and unbounded)

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
basis  = mnt.get_basis_signal_to_moments(phases)

(compressed_bounded_w_upper_moment_image,
 relative_scale,
 mins_scaled, maxs_scaled,
 global_max) = mnt.bounded_w_upper_forward(basis, spectral_image)

## Decompression

### Current (bounded case)

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
inv_basis = mnt.get_basis_moment_to_signal(phases)

d_bounded_spectral_image = mnt.bounded_backward(inv_basis, compressed_bounded_moment_image, mins, maxs)

In [ ]:
show_spectral_reflective_images(
    cmf_cie_2006_2, illu_d65, 
    ['original', 'bounded'],
    [image_in.reflective_wavelengths_nm, wavelengths],
    [image_in.reflective_image, d_bounded_spectral_image])

### Current (unbounded case)

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
inv_basis = mnt.get_basis_moment_to_signal(phases)

d_unbounded_spectral_image = mnt.bounded_backward(inv_basis, compressed_unbounded_moment_image, mins, maxs)

In [ ]:
show_spectral_reflective_images(
    cmf_cie_2006_2, illu_d65, 
    ['original', 'unbounded'],
    [image_in.reflective_wavelengths_nm, wavelengths],
    [image_in.reflective_image, d_unbounded_spectral_image])

### Current (unbounded to bounded case)

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
inv_basis = mnt.get_basis_moment_to_signal(phases)

d_unbounded_to_bounded_spectral_image = mnt.bounded_backward(inv_basis, compressed_unbounded_to_bounded_moment_image, mins, maxs)

In [ ]:
show_spectral_reflective_images(
    cmf_cie_2006_2, illu_d65, 
    ['original', 'unbounded to bounded'],
    [image_in.reflective_wavelengths_nm, wavelengths],
    [image_in.reflective_image, d_unbounded_to_bounded_spectral_image])

### Alternative (bounded and unbounded)

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
inv_basis = mnt.get_basis_moment_to_signal(phases)

d_bounded_w_upper_w_upper_spectral_image = mnt.bounded_w_upper_backward(
    inv_basis,
    compressed_bounded_w_upper_moment_image, relative_scale,
    mins_scaled, maxs_scaled,
    global_max)

In [ ]:
org_rgb_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    image_in.reflective_wavelengths_nm, image_in.reflective_image
)

d_rgb_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    wavelengths, d_bounded_w_upper_w_upper_spectral_image
)

fig, ax = plt.subplots(1, 2)

ax[0].set_title('Original image')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].imshow(org_rgb_image**(1./2.2))

ax[1].set_title('Decompressed image')
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].imshow(d_rgb_image**(1./2.2))

fig.tight_layout()

plt.show()

## Comparison of quantization curves

In [ ]:
import compress.quantization as quant

In [ ]:
q_curve_bounded              =              quant.bounded_generate_quantization_curve(np.array(wavelengths), spectral_image, 14)
q_curve_unbounded            =            quant.unbounded_generate_quantization_curve(np.array(wavelengths), spectral_image, 14)
q_curve_unbounded_to_bounded = quant.unbounded_to_bounded_generate_quantization_curve(np.array(wavelengths), spectral_image, 12)
q_curve_bounded_w_upper      =      quant.bounded_w_upper_generate_quantization_curve(np.array(wavelengths), spectral_image, 12)

In [ ]:
fig, ax = plt.subplots(1, 1)

a = np.arange(len(wavelengths))
ax.plot(a, q_curve_bounded, label='Bounded')
ax.plot(a, q_curve_unbounded, label='Unbounded')
ax.plot(a, q_curve_unbounded_to_bounded, label='Unbounded to Bounded')
ax.plot(a, q_curve_bounded_w_upper, label='New')

ax.legend()

plt.show()

In [ ]:
print('Bounded signal:', quant.bounded_err_for_curve(np.array(wavelengths), spectral_image, q_curve_bounded))
print('Unbounded:', quant.unbounded_err_for_curve(np.array(wavelengths), spectral_image, q_curve_unbounded))
print('Bounded with upper:', quant.bounded_w_upper_err_for_curve(np.array(wavelengths), spectral_image, q_curve_bounded_w_upper))

In [ ]:
phases = mnt.wavelengths_to_phase(np.array(wavelengths))
inv_basis = mnt.get_basis_moment_to_signal(phases)

q_compressed_moment_image        = quant.quantize_dequantize(compressed_moment_image, 8, 6)
q_compressed_scaled_moment_image = quant.quantize_dequantize(compressed_scaled_moment_image, 8, 6)

d_bounded_spectral_image = mnt.bounded_backward(
    inv_basis,
    q_compressed_moment_image,
    mins, maxs)

d_bounded_w_upper_spectral_image = mnt.bounded_w_upper_backward(
    inv_basis,
    q_compressed_scaled_moment_image, relative_scale,
    mins_scaled, maxs_scaled,
    global_max)

org_rgb_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    image_in.reflective_wavelengths_nm, image_in.reflective_image
)

d_rgb_bounded_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    wavelengths, d_bounded_spectral_image
)

d_rgb_bounded_w_upper_image = cmf_cie_2006_2.get_sRGB_lin_reflective_img(
    illu_d65.data[:,0], illu_d65.data[:,1],
    wavelengths, d_bounded_w_upper_spectral_image
)

fig, ax = plt.subplots(1, 3)

ax[0].set_title('Original image')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].imshow(org_rgb_image**(1./2.2))

ax[1].set_title('Decompressed imag bounded')
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].imshow(d_rgb_bounded_image**(1./2.2))


ax[2].set_title('Decompressed image bounded w upper')
ax[2].set_xticks([])
ax[2].set_yticks([])
ax[2].imshow(d_rgb_bounded_w_upper_image**(1./2.2))

fig.tight_layout()

plt.show()